In [6]:
import pandas as pd

In [15]:
df = pd.read_csv("pycharm/specs/lookup_codes_1.csv", dtype={"code": str, "code_2":str}, encoding="utf-8")
df

,code_2,code,desc,code_base,desc_base
0,1,1,Live Animals; Animal Products,F,Food and feed
1,2,1,Live Animals; Animal Products,F,Food and feed
2,3,1,Live Animals; Animal Products,F,Food and feed
3,4,1,Live Animals; Animal Products,F,Food and feed
4,5,1,Live Animals; Animal Products,F,Food and feed
5,6,2,Vegetable Products,F,Food and feed
6,7,2,Vegetable Products,F,Food and feed
7,8,2,Vegetable Products,F,Food and feed
8,9,2,Vegetable Products,F,Food and feed
9,10,2,Vegetable Products,F,Food and feed


In [10]:
#Now create this for the real data
import sqlite3
conn = sqlite3.connect(r"pycharm\trade_data.db")


In [12]:
sql = """
select 
code8,
code6,
code4,
code2,
l1.code as code1,
l1.code_base as code0,
mk_commodity_alpha_all as desc8,
l6.desc as desc6, 
l4.desc as desc4, 
l2.desc as desc2, 
l1.desc as desc1,
l1.desc_base as code0,
quantity from 

(select product_code as code8, 
substr(product_code,1,6) as code6,
substr(product_code,1,4) as code4,
substr(product_code,1,2) as code2,

sum(quantity) as quantity
from der_imports_country_products_port_month_8
where year = 2014
group by product_code) as s

left join eightdigitcodes as l8
on s.code8 = l8.mk_comcode8

left join lookup_codes_6 as l6
on s.code6 = l6.code

left join lookup_codes_4 as l4
on s.code4 = l4.code

left join lookup_codes_2 as l2
on s.code2 = l2.code

left join lookup_codes_1 as l1
on s.code2 = l1.code_2
"""

df_all = pd.read_sql(sql, conn)
df_all.head()

,code8,code6,code4,code2,code1,code0,desc8,desc6,desc4,desc2,desc1,code0,quantity
0,01012100,010121,0101,01,None,None,Pure-bred breeding horses,Pure-bred breeding horses,"Live horses, asses, mules and hinnies",Live animals,None,None,45592174
1,01012990,010129,0101,01,None,None,"Live horses (excl. for slaughter, pure-bred fo...",Live horses (excl. pure-bred for breeding),"Live horses, asses, mules and hinnies",Live animals,None,None,2498001
2,01022110,010221,0102,01,None,None,"Pure-bred breeding heifers ""female bovines tha...",Pure-bred cattle for breeding,Live bovine animals,Live animals,None,None,49329
3,01022130,010221,0102,01,None,None,Pure-bred breeding cows (excl. heifers),Pure-bred cattle for breeding,Live bovine animals,Live animals,None,None,2784
4,01031000,010310,0103,01,None,None,Pure-bred breeding swine,Pure-bred breeding swine,Live swine,Live animals,None,None,120277


In [3]:
def find_element(children_list,name):
    """
    Find element in children list
    if exists or return none
    """
    for i in children_list:
        if i["name"] == name:
            return i
    #If not found return None
    return None

def add_node(levels,row, value_field, nest):
    """
    The path is a list.  Each element is a name that corresponds 
    to a level in the final nested dictionary.  
    """

    #Get first name from level
    this_level = levels.pop(0)
    this_name = row[this_level["name"]]
    
    level_properties = this_level["properties"]
    
    #Does the element exist already?
    element = find_element(nest["children"], this_name)

    #If the element exists, we can use it, otherwise we need to create a new one
    if element:


        if len(levels)>0:
            add_node(levels, row, value_field, element)
        
    #Else it does not exist so create it and return its children
    else:
        new_child ={"name":this_name}
        for prop, rowkey in level_properties.iteritems():
            new_child[prop] = row[rowkey]
        
        if len(levels) == 0:

            new_child["value"] = row[value_field]
            nest["children"].append(new_child)
        else:
            #Add new element

            new_child["children"] = []
            nest["children"].append(new_child)
            
            #Get added element 
            element = nest["children"][-1]
            
            #Still elements of path left so recurse
            add_node(levels, row, value_field, element)
    

In [4]:
d_all = {"name": "root",
         "desc": "*",
"children": []}


my_levels = [{"name": "code1",
           "properties" : {"desc":"desc1"}},
         {"name": "code2",
           "properties" : {"desc":"desc2"}},
         {"name": "code4",
           "properties" : {"desc":"desc4"}},
         {"name": "code6",
           "properties" : {"desc":"desc6"}},
         {"name": "code8",
           "properties" : {"desc":"desc8"}}]
           
for row in df_all.iterrows():
    r = row[1]
    value_field = "quantity"
    levels_info = [i for i in my_levels]
    add_node(levels_info, r, value_field,d_all)

NameError: name 'df_all' is not defined

In [5]:
import json
print json.dumps(d_all,  sort_keys=False, indent=2)

{
  "children": [], 
  "name": "root", 
  "desc": "*"
}


8181

In [110]:
import json
with open('data2.txt', 'w') as outfile:
    #json.dump(d_all, outfile, sort_keys=False, indent=2)
    json.dump(d_all, outfile)


In [115]:
df_all[pd.isnull(df_all["code1"])]

,code8,code6,code4,code2,code1,desc8,desc6,desc4,desc2,desc1,quantity
8159,99005291,990052,9900,99,None,None,None,None,None,None,12475940
8160,99025278,990252,9902,99,None,None,None,None,None,None,2000475
8161,99028522,990285,9902,99,None,None,None,None,None,None,51851645
8162,99028525,990285,9902,99,None,None,None,None,None,None,475880746
8163,99029511,990295,9902,99,None,None,None,None,None,None,10659
8164,99029512,990295,9902,99,None,None,None,None,None,None,186737535
8165,99029513,990295,9902,99,None,None,None,None,None,None,92439697
8166,99029515,990295,9902,99,None,None,None,None,None,None,27283371
8167,99036593,990365,9903,99,None,None,None,None,None,None,18270750
8168,99039575,990395,9903,99,None,None,None,None,None,None,52202409


In [10]:
df = df_all.head().copy()

In [112]:
import json
with open('data.txt', 'w') as outfile:
    #json.dump(d_all, outfile, sort_keys=False, indent=2)
    json.dump(d_all, outfile)


In [13]:
md = {"a":1}
for k, v in md.iteritems():
    print k
    print v

a
1


In [63]:
import copy

In [65]:
md.copy()

{'a': 1}

In [66]:
a= [1,2,3]

In [67]:
b = [i for i in a]

In [69]:
b.pop()

3

In [70]:
b

[1, 2]

In [71]:
a

[1, 2, 3]